Model

In [ ]:
class CNN_model(nn.Module):
    
    def __init__(self, kernel_size_1, kernel_size_2, depth_CL, neurons_MLP, drop_out, neurons_MLP_D):
        
        super().__init__()

        # 1. Convolutional Layer 1
        self.conv2Dblock1=nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=depth_CL, kernel_size=kernel_size_1, stride=1, padding=1),
            nn.BatchNorm2d(depth_CL),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(p=drop_out),
        )

        # 2. Convolutional Layer 2
        self.conv2Dblock2=nn.Sequential(
            nn.Conv2d(in_channels=depth_CL, out_channels=depth_CL, kernel_size=kernel_size_2, stride=1, padding=1),
            nn.BatchNorm2d(depth_CL),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Dropout(p=drop_out),
        )

        # 3. Fully Connected Layer 1
        self.MLPblock1=nn.Sequential(
            
            nn.Linear(in_features=depth_CL*7*7, out_features=neurons_MLP),
            nn.ReLU(),
            nn.Dropout(p=drop_out),
        )



        